In [6]:
# import the Printf module for formatted output
using Printf

# Logika find_macheps(T):
    1. Zaczynamy z epsilon = T(1.0).
    2. Dzielimy go na dwa (epsilon / 2.0).
    3. Sprawdzamy, czy T(1.0) + (epsilon / 2.0) jest wciąż większe niż T(1.0).
    4. Jeśli tak, to znaczy, że epsilon / 2.0 jest jeszcze "zauważalne" i możemy ustawić epsilon = epsilon / 2.0.
    5. Jeśli nie (czyli 1.0 + epsilon / 2.0 == 1.0), to epsilon / 2.0 zostało zaokrąglone w dół. Oznacza to, że ostatnia działająca wartość (aktualny epsilon) jest tym, czego szukamy.

In [7]:
"""
find_macheps(T)

Wyznacza iteracyjnie epsilon maszynowy (macheps) dla typu T.
Jest to najmniejsza liczba 'eps', dla której 1.0 + eps > 1.0.
"""
function find_macheps(T::Type{<:AbstractFloat})
    one = T(1.0) # Castujemy 1.0 do typu T
    two = T(2.0) 
    
    epsilon = one

    # Pętla wykonuje się, dopóki (epsilon / 2.0) dodane do 1.0
    # wciąż daje wynik większy niż 1.0
    while one + (epsilon / two) > one
        epsilon = epsilon / two
    end
    
    return epsilon
end

# --- Testowanie i porównanie ---

println("===== Epsilon Maszynowy (macheps) =====")
types_to_test = [Float16, Float32, Float64]

for T in types_to_test
    calculated_eps = find_macheps(T)
    builtin_eps = eps(T) # eps(T) jest równoważne eps(one(T))

    println("----------------------------------------")
    println("Typ: $T")
    @printf("  Obliczony iteracyjnie: %.10e\n", calculated_eps)
    @printf("  Wartość z eps(%s):   %.10e\n", T, builtin_eps)
    println("  Czy wartości są równe: $(calculated_eps == builtin_eps)")
end

===== Epsilon Maszynowy (macheps) =====
----------------------------------------
Typ: Float16
  Obliczony iteracyjnie: 9.7656250000e-04
  Wartość z eps(Float16):   9.7656250000e-04
  Czy wartości są równe: true
----------------------------------------
Typ: Float32
  Obliczony iteracyjnie: 1.1920928955e-07
  Wartość z eps(Float32):   1.1920928955e-07
  Czy wartości są równe: true
----------------------------------------
Typ: Float64
  Obliczony iteracyjnie: 2.2204460493e-16
  Wartość z eps(Float64):   2.2204460493e-16
  Czy wartości są równe: true


# Logika find_eta(T):
    1. Zaczynamy z eta = T(1.0).
    2. Sprawdzamy, czy eta / 2.0 jest wciąż $> 0.0`.
    3. Jeśli tak, kontynuujemy pętlę z eta = eta / 2.0.
    4. Gdy eta / 2.0 da 0.0 (przez underflow), pętla się zatrzyma, a ostatnia zapisana wartość eta będzie najmniejszą liczbą subnormalną.

In [ ]:
"""
find_eta(T)

Wyznacza iteracyjnie najmniejszą dodatnią liczbę maszynową 'eta' (MIN_sub)
dla typu T, zaczynając od 1.0 i dzieląc przez 2.0.
"""
function find_eta(T::Type{<:AbstractFloat})
    two = T(2.0)
    zero = T(0.0)
    
    eta = T(1.0)
    
    # Dzielimy, dopóki następny krok (eta / 2.0) wciąż jest większy od zera
    while (eta / two) > zero
        eta = eta / two
    end
    
    return eta
end

# --- Testowanie i porównanie ---

println("\n===== Najmniejsza liczba dodatnia (eta) =====")

for T in types_to_test
    calculated_eta = find_eta(T)
    
    # nextfloat(T(0.0)) to wbudowana funkcja Julii
    # dająca kolejną liczbę maszynową po 0.0, czyli eta
    builtin_eta = nextfloat(T(0.0))

    println("----------------------------------------")
    println("Typ: $T")
    @printf("  Obliczona iteracyjnie: %.10e\n", calculated_eta)
    @printf("  Wartość z nextfloat:   %.10e\n", builtin_eta)
    println("  Czy wartości są równe: $(calculated_eta == builtin_eta)")
end


===== Najmniejsza liczba dodatnia (eta) =====
----------------------------------------
Typ: Float16
  Obliczona iteracyjnie: 5.9604644775e-08
  Wartość z nextfloat:   5.9604644775e-08
  Czy wartości są równe: true
----------------------------------------
Typ: Float32
  Obliczona iteracyjnie: 1.4012984643e-45
  Wartość z nextfloat:   1.4012984643e-45
  Czy wartości są równe: true
----------------------------------------
Typ: Float64
  Obliczona iteracyjnie: 4.9406564584e-324
  Wartość z nextfloat:   4.9406564584e-324
  Czy wartości są równe: true



# Logika find_max(T):
    1. Znajdź największą potęgę dwójki. Zaczynamy od 1.0 i mnożymy przez 2.0 tak długo, aż kolejna wartość będzie Inf. Zapisujemy ostatnią skończoną potęgę dwójki (nazwijmy ją max_pow2).
    2. Wypełnij mantysę. max_pow2 to liczba z mantysą 1.000... i maksymalnym wykładnikiem. Prawdziwy max ma mantysę 1.111.... Musimy "iteracyjnie" dodać bity.
    3. Zaczynamy od max_val = max_pow2.
    4. Bierzemy increment = max_pow2 / 2.0.
    5. W pętli: próbujemy dodać increment do max_val.
    6. Jeśli max_val + increment nie jest Inf, to znaczy, że ten bit się "zmieścił" -> aktualizujemy max_val.
    7. Jeśli max_val + increment jest już Inf LUB increment stał się tak mały, że max_val + increment == max_val (stracił precyzję), przerywamy.
    8. Dzielimy increment przez 2 i powtarzamy.

In [9]:
"""
find_max(T)

Wyznacza iteracyjnie największą skończoną liczbę maszynową (MAX)
dla typu T.
"""
function find_max(T::Type{<:AbstractFloat})
    one = T(1.0)
    two = T(2.0)
    
    # --- Część 1: Znajdź największą potęgę 2.0 ---
    max_pow2 = one
    last_finite_pow2 = one
    
    while !isinf(max_pow2)
        last_finite_pow2 = max_pow2
        max_pow2 = max_pow2 * two
    end
    
    # last_finite_pow2 to teraz 2^E_max
    max_val = last_finite_pow2
    
    # --- Część 2: "Wypełnij" bity mantysy ---
    # Zaczynamy od dodawania 2^(E_max - 1)
    increment = last_finite_pow2 / two
    
    while increment > T(0.0)
        # Sprawdzamy, czy dodanie bitu nie powoduje nieskończoności
        # ORAZ czy dodanie bitu w ogóle coś zmienia 
        # (jeśli 'increment' jest za mały, 'max_val + increment' == 'max_val')
        if !isinf(max_val + increment) && (max_val + increment) != max_val
            max_val = max_val + increment
        else
            # Jeśli increment jest już za mały, by cokolwiek zmienić, 
            # nie ma sensu próbować z mniejszymi wartościami.
            break
        end
        increment = increment / two
    end
    
    return max_val
end

# --- Testowanie i porównanie ---

println("\n===== Największa liczba skończona (MAX) =====")

for T in types_to_test
    calculated_max = find_max(T)
    builtin_max = floatmax(T)

    println("----------------------------------------")
    println("Typ: $T")
    @printf("  Obliczona iteracyjnie: %.10e\n", calculated_max)
    @printf("  Wartość z floatmax:    %.10e\n", builtin_max)
    println("  Czy wartości są równe: $(calculated_max == builtin_max)")
end


===== Największa liczba skończona (MAX) =====
----------------------------------------
Typ: Float16
  Obliczona iteracyjnie: 6.5504000000e+04
  Wartość z floatmax:    6.5504000000e+04
  Czy wartości są równe: true
----------------------------------------
Typ: Float32
  Obliczona iteracyjnie: 3.4028234664e+38
  Wartość z floatmax:    3.4028234664e+38
  Czy wartości są równe: true
----------------------------------------
Typ: Float64
  Obliczona iteracyjnie: 1.7976931349e+308
  Wartość z floatmax:    1.7976931349e+308
  Czy wartości są równe: true


# porównanie z float.h (C)
## Float64 (C: double): Wartość DBL_MAX to 1.7976931348623158e+308. Zgadza się.
## Float32 (C: float): Wartość FLT_MAX to 3.40282347e+38F. Zgadza się.
## Float16 (C: _Float16): Wartość FLT16_MAX to 6.55040000e+04. Zgadza się.